In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from scipy import stats

nltk.download('vader_lexicon')
data = pd.read_csv("data_trial2.csv", encoding = "utf-8")
pd.options.display.max_columns = None

header = list(data)
header.remove("start_datetime")
header.remove("end_datetime_trial1")
header.remove("end_datetime_trial2")
#data cleaning
#making everything to_lower_case
for (index,row) in data.iterrows():
    for h in header:   
        if(type(row[h]) == str):
            data.loc[index,h] = row[h].strip().lower()


        



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
#accuracy for each participate (strictest--> just based on best category)
trial = "exp2_trial"
best = "_best_category"
second = "_second_best_category"
palette = "_palette"
accuracy = {}
second_accuracy = {}

for (index, row) in data.iterrows():
    accuracy_counter_strict = 0
    accuracy_counter_med = 0
    for i in range(5):
        pal = row[trial+str(i)+palette]
        if (row[trial+str(i)+best] == pal):
            accuracy_counter_strict += 1
        
        if (row[trial+str(i)+best] == pal or row[trial+str(i)+second] == pal):
            accuracy_counter_med +=1 
    accuracy[index] = float(accuracy_counter_strict) / 5
    second_accuracy[index] = float(accuracy_counter_med) / 5

accuracy
data["strict_accuracy"] = [accuracy[index] for index, row in data.iterrows()]
data["med_accuracy"] = [second_accuracy[index] for index, row in data.iterrows()]


data
    


    


,id,firstname,lastname,age,major_occ,graphic_design_exp,artist_exp,start_datetime,end_datetime_trial1,end_datetime_trial2,exp2_trial0_curr_emotion,exp2_trial0_curr_emotion_strength,exp2_trial0_emotion_0,exp2_trial0_emotion_1,exp2_trial0_emotion_2,exp2_trial0_emotion_3,exp2_trial0_emotion_4,exp2_trial0_best_category,exp2_trial0_best_category_strength,exp2_trial0_second_best_category,exp2_trial0_second_best_category_strength,exp2_trial0_perception,exp2_trial0_has_influence,exp2_trial0_how_influence,exp2_trial0_light,exp2_trial0_usage,exp2_trial0_usage_strength,exp2_trial0_time,exp2_trial0_imageName,exp2_trial0_palette,exp2_trial1_curr_emotion,exp2_trial1_curr_emotion_strength,exp2_trial1_emotion_0,exp2_trial1_emotion_1,exp2_trial1_emotion_2,exp2_trial1_emotion_3,exp2_trial1_emotion_4,exp2_trial1_best_category,exp2_trial1_best_category_strength,exp2_trial1_second_best_category,exp2_trial1_second_best_category_strength,exp2_trial1_perception,exp2_trial1_has_influence,exp2_trial1_how_influence,exp2_trial1_light,exp2_trial1_usage,exp2_trial1_usage_strength,exp2_trial1_time,exp2_trial1_imageName,exp2_trial1_palette,exp2_trial2_curr_emotion,exp2_trial2_curr_emotion_strength,exp2_trial2_emotion_0,exp2_trial2_emotion_1,exp2_trial2_emotion_2,exp2_trial2_emotion_3,exp2_trial2_emotion_4,exp2_trial2_best_category,exp2_trial2_best_category_strength,exp2_trial2_second_best_category,exp2_trial2_second_best_category_strength,exp2_trial2_perception,exp2_trial2_has_influence,exp2_trial2_how_influence,exp2_trial2_light,exp2_trial2_usage,exp2_trial2_usage_strength,exp2_trial2_time,exp2_trial2_imageName,exp2_trial2_palette,exp2_trial3_curr_emotion,exp2_trial3_curr_emotion_strength,exp2_trial3_emotion_0,exp2_trial3_emotion_1,exp2_trial3_emotion_2,exp2_trial3_emotion_3,exp2_trial3_emotion_4,exp2_trial3_best_category,exp2_trial3_best_category_strength,exp2_trial3_second_best_category,exp2_trial3_second_best_category_strength,exp2_trial3_perception,exp2_trial3_has_influence,exp2_trial3_how_influence,exp2_trial3_light,exp2_trial3_usage,exp2_trial3_usage_strength,exp2_trial3_time,exp2_trial3_imageName,exp2_trial3_palette,exp2_trial4_curr_emotion,exp2_trial4_curr_emotion_strength,exp2_trial4_emotion_0,exp2_trial4_emotion_1,exp2_trial4_emotion_2,exp2_trial4_emotion_3,exp2_trial4_emotion_4,exp2_trial4_best_category,exp2_trial4_best_category_strength,exp2_trial4_second_best_category,exp2_trial4_second_best_category_strength,exp2_trial4_perception,exp2_trial4_has_influence,exp2_trial4_how_influence,exp2_trial4_light,exp2_trial4_usage,exp2_trial4_usage_strength,exp2_trial4_time,exp2_trial4_imageName,exp2_trial4_palette,strict_accuracy,med_accuracy
0,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",serious,1,calm,serious,unexcited,relaxed,trustworthy,calm,NaN,serious,4,it is easy on the eyes and makes the infograph...,y,readibility,1,0,3,209.829,introverts,calm,serious,1,bold,calm,relaxed,trustworthy,positive,trustworthy,NaN,serious,3,a darker palette makes it a bit more serious t...,y,tone,1,2,3,585.043,commodities,trustworthy,calm,1,calm,relaxed,positive,uplifted,playful,positive,NaN,playful,3,"yes, the lighter background and text are more ...",y,tone,2,0,4,668.179,shopping,positive,serious,1,serious,bland,bored,negative,unexcited,serious,NaN,negative,2,the mostly-gray palette makes the infographic ...,y,visual_interest,0,1,4,785.707,burger,negative,serious,3,serious,negative,bland,bored,unexcited,NaN,serious,negative,3,"similar to the last one, this is a much more b...",y,tone,0,1,3,837.279,gaming,negative,0.6,1.0
1,5cc8efc56765220004bf4fc0,marcelo,guillí©n,53,electrical engineer,1,1,"Wed, 01 May 2019 00:29:44 GMT","Wed, 01 May 2019 00:33:19 GMT","Wed, 01 May 2019 01:00:53 GMT",uninterested,1,boring,simple,sad,calm,conflicted,negative,NaN,serious,3,yes. it is not appalling and not interested,y,visual_interest,1,1,4,645.540,introverts,calm,contrast,1,

In [12]:
#using NLTK's sentiment analysis to determine whether a paletteName is considered positive or negative
#if someone gets the right category, we'll give it to them!

paletteNames = ["disturbing", "negative", "serious", "trustworthy", "calm", "playful", "positive", "exciting"];
pos_pal = []
neg_pal = []
neu_pal = [] #know ahead of time that this will be empty, but coding for generalizability
sid = SIA()
trial = "exp2_trial"
best = "_best_category"
second = "_second_best_category"
palette = "_palette"
first_accuracy = {}
second_accuracy = {}


def isNeg(val):
    if val in neg_pal:
        return True
    else:
        return False
    
for p in paletteNames:
    score = sid.polarity_scores(p)
    if(score['neg'] == 1.0):
        neg_pal.append(p)
    elif(score['neu'] == 1.0):
        neu_pal.append(p)
    elif(score['pos'] == 1.0):
        pos_pal.append(p)


for (index, row) in data.iterrows():
    best_acc_count = 0
    second_acc_count = 0
    for i in range(5):
        pal = row[trial+str(i)+palette]
        best_val = row[trial+str(i)+best]
        second_val = row[trial+str(i)+second]

        if ((isNeg(pal) == isNeg(best_val)) or (isNeg(pal) == isNeg(second_val))) : #should return true if they both match
            second_acc_count += 1
            
        if(isNeg(pal) == isNeg(best_val)):
            best_acc_count +=1
        
    first_accuracy[index] = float(best_acc_count) / 5
    second_accuracy[index] = float(second_acc_count) / 5

data["best_valence_accuracy"] = [first_accuracy[index] for index, row in data.iterrows()]
data["med_valence_accuracy"] = [second_accuracy[index] for index, row in data.iterrows()]

data
    


   


,id,firstname,lastname,age,major_occ,graphic_design_exp,artist_exp,start_datetime,end_datetime_trial1,end_datetime_trial2,exp2_trial0_curr_emotion,exp2_trial0_curr_emotion_strength,exp2_trial0_emotion_0,exp2_trial0_emotion_1,exp2_trial0_emotion_2,exp2_trial0_emotion_3,exp2_trial0_emotion_4,exp2_trial0_best_category,exp2_trial0_best_category_strength,exp2_trial0_second_best_category,exp2_trial0_second_best_category_strength,exp2_trial0_perception,exp2_trial0_has_influence,exp2_trial0_how_influence,exp2_trial0_light,exp2_trial0_usage,exp2_trial0_usage_strength,exp2_trial0_time,exp2_trial0_imageName,exp2_trial0_palette,exp2_trial1_curr_emotion,exp2_trial1_curr_emotion_strength,exp2_trial1_emotion_0,exp2_trial1_emotion_1,exp2_trial1_emotion_2,exp2_trial1_emotion_3,exp2_trial1_emotion_4,exp2_trial1_best_category,exp2_trial1_best_category_strength,exp2_trial1_second_best_category,exp2_trial1_second_best_category_strength,exp2_trial1_perception,exp2_trial1_has_influence,exp2_trial1_how_influence,exp2_trial1_light,exp2_trial1_usage,exp2_trial1_usage_strength,exp2_trial1_time,exp2_trial1_imageName,exp2_trial1_palette,exp2_trial2_curr_emotion,exp2_trial2_curr_emotion_strength,exp2_trial2_emotion_0,exp2_trial2_emotion_1,exp2_trial2_emotion_2,exp2_trial2_emotion_3,exp2_trial2_emotion_4,exp2_trial2_best_category,exp2_trial2_best_category_strength,exp2_trial2_second_best_category,exp2_trial2_second_best_category_strength,exp2_trial2_perception,exp2_trial2_has_influence,exp2_trial2_how_influence,exp2_trial2_light,exp2_trial2_usage,exp2_trial2_usage_strength,exp2_trial2_time,exp2_trial2_imageName,exp2_trial2_palette,exp2_trial3_curr_emotion,exp2_trial3_curr_emotion_strength,exp2_trial3_emotion_0,exp2_trial3_emotion_1,exp2_trial3_emotion_2,exp2_trial3_emotion_3,exp2_trial3_emotion_4,exp2_trial3_best_category,exp2_trial3_best_category_strength,exp2_trial3_second_best_category,exp2_trial3_second_best_category_strength,exp2_trial3_perception,exp2_trial3_has_influence,exp2_trial3_how_influence,exp2_trial3_light,exp2_trial3_usage,exp2_trial3_usage_strength,exp2_trial3_time,exp2_trial3_imageName,exp2_trial3_palette,exp2_trial4_curr_emotion,exp2_trial4_curr_emotion_strength,exp2_trial4_emotion_0,exp2_trial4_emotion_1,exp2_trial4_emotion_2,exp2_trial4_emotion_3,exp2_trial4_emotion_4,exp2_trial4_best_category,exp2_trial4_best_category_strength,exp2_trial4_second_best_category,exp2_trial4_second_best_category_strength,exp2_trial4_perception,exp2_trial4_has_influence,exp2_trial4_how_influence,exp2_trial4_light,exp2_trial4_usage,exp2_trial4_usage_strength,exp2_trial4_time,exp2_trial4_imageName,exp2_trial4_palette,strict_accuracy,med_accuracy,best_valence_accuracy,med_valence_accuracy
0,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",serious,1,calm,serious,unexcited,relaxed,trustworthy,calm,NaN,serious,4,it is easy on the eyes and makes the infograph...,y,readibility,1,0,3,209.829,introverts,calm,serious,1,bold,calm,relaxed,trustworthy,positive,trustworthy,NaN,serious,3,a darker palette makes it a bit more serious t...,y,tone,1,2,3,585.043,commodities,trustworthy,calm,1,calm,relaxed,positive,uplifted,playful,positive,NaN,playful,3,"yes, the lighter background and text are more ...",y,tone,2,0,4,668.179,shopping,positive,serious,1,serious,bland,bored,negative,unexcited,serious,NaN,negative,2,the mostly-gray palette makes the infographic ...,y,visual_interest,0,1,4,785.707,burger,negative,serious,3,serious,negative,bland,bored,unexcited,NaN,serious,negative,3,"similar to the last one, this is a much more b...",y,tone,0,1,3,837.279,gaming,negative,0.6,1.0,0.8,1.0
1,5cc8efc56765220004bf4fc0,marcelo,guillí©n,53,electrical engineer,1,1,"Wed, 01 May 2019 00:29:44 GMT","Wed, 01 May 2019 00:33:19 GMT","Wed, 01 May 2019 01:00:53 GMT",uninterested,1,boring,simple,sad,calm,conflicted,negative,NaN,serious,3,yes. it is not appalling and not interested,y,visual

In [13]:
#per trial accuracy --> maybe people got better over time? 
acc_trial = {}
for i in range(5):
    arr = data[trial+str(i) +best] == data[trial+str(i)+palette]
    count = 0
    for a in arr:
        if (a):
            count+=1
    acc_trial[i] = count/len(arr)
acc_trial

{0: 0.16129032258064516,
 1: 0.12903225806451613,
 2: 0.22580645161290322,
 3: 0.25806451612903225,
 4: 0.0}

In [14]:
# to do palette group by analysis, I'm going to have to rearrange the dataframe
df = []
exp = "exp2_trial"
t_head = ["curr_emotion", "curr_emotion_strength", "emotion_0", "emotion_1", "emotion_2", "emotion_3", "emotion_4", "best_category", "best_category_strength", "second_best_category", "second_best_category_strength", "perception", "has_influence", "how_influence", "light", "usage", "usage_strength", "time", "imageName", "palette"]
trial_headers = ["trialNum", "index"]
other_data = ["id", "firstname", "lastname", "age", "major_occ", "graphic_design_exp", "artist_exp", "start_datetime", "end_datetime_trial1", "end_datetime_trial2", "best_valence_accuracy", "med_valence_accuracy", "strict_accuracy", "med_accuracy"]
#palette_df[other_data]
#trial_headers = ["trialNum"] + list(range(20)) + ["index"]
slice_len = 20
last_index = 10
counter = 0
for (index, row) in data.iterrows():
    begin = row[other_data]
    last_index = 10
    for i in range(5):
        b = begin
        begin["trialNum"] = i
        arr = row[last_index: last_index + slice_len].values
        #print(arr)
        join = b.append(pd.Series(arr, index = t_head))
        join["index"] = counter
        df.append(join)
        last_index += slice_len
        counter+=1
    

palette_df = pd.DataFrame(data = df, columns=other_data+ ["trialNum"] + t_head + ["index"])
palette_df.head()

    


        
       
    

,id,firstname,lastname,age,major_occ,graphic_design_exp,artist_exp,start_datetime,end_datetime_trial1,end_datetime_trial2,best_valence_accuracy,med_valence_accuracy,strict_accuracy,med_accuracy,trialNum,curr_emotion,curr_emotion_strength,emotion_0,emotion_1,emotion_2,emotion_3,emotion_4,best_category,best_category_strength,second_best_category,second_best_category_strength,perception,has_influence,how_influence,light,usage,usage_strength,time,imageName,palette,index
0,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,0,serious,1,calm,serious,unexcited,relaxed,trustworthy,calm,NaN,serious,4,it is easy on the eyes and makes the infograph...,y,readibility,1,0,3,209.829,introverts,calm,0
1,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,1,serious,1,bold,calm,relaxed,trustworthy,positive,trustworthy,NaN,serious,3,a darker palette makes it a bit more serious t...,y,tone,1,2,3,585.043,commodities,trustworthy,1
2,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,2,calm,1,calm,relaxed,positive,uplifted,playful,positive,NaN,playful,3,"yes, the lighter background and text are more ...",y,tone,2,0,4,668.179,shopping,positive,2
3,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,3,serious,1,serious,bland,bored,negative,unexcited,serious,NaN,negative,2,the mostly-gray palette makes the infographic ...,y,visual_interest,0,1,4,785.707,burger,negative,3
4,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,4,serious,3,serious,negative,bland,bored,unexcited,NaN,serious,negative,3,"similar to the last one, this is a much more b...",y,tone,0,1,3,837.279,gaming,negative,4


In [15]:
#average valence for emotions responsed 
avg_compound = []
trial = "exp2_trial"
best = "_best_category"
second = "_second_best_category"
palette = "_palette"
for index, row in palette_df.iterrows():
    counter = 0.0
    for i in range(5):
        emo = "emotion_" +str(i)
        #print(str(row[emo]))
        val = sid.polarity_scores(str(row[emo]))["compound"]
        counter = counter + val
    #print(counter)
    avg_compound.append(float(counter)/5)
palette_df["emotion_valence_avg"] = avg_compound
palette_df.head()



,id,firstname,lastname,age,major_occ,graphic_design_exp,artist_exp,start_datetime,end_datetime_trial1,end_datetime_trial2,best_valence_accuracy,med_valence_accuracy,strict_accuracy,med_accuracy,trialNum,curr_emotion,curr_emotion_strength,emotion_0,emotion_1,emotion_2,emotion_3,emotion_4,best_category,best_category_strength,second_best_category,second_best_category_strength,perception,has_influence,how_influence,light,usage,usage_strength,time,imageName,palette,index,emotion_valence_avg
0,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,0,serious,1,calm,serious,unexcited,relaxed,trustworthy,calm,NaN,serious,4,it is easy on the eyes and makes the infograph...,y,readibility,1,0,3,209.829,introverts,calm,0,0.25846
1,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,1,serious,1,bold,calm,relaxed,trustworthy,positive,trustworthy,NaN,serious,3,a darker palette makes it a bit more serious t...,y,tone,1,2,3,585.043,commodities,trustworthy,1,0.46174
2,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,2,calm,1,calm,relaxed,positive,uplifted,playful,positive,NaN,playful,3,"yes, the lighter background and text are more ...",y,tone,2,0,4,668.179,shopping,positive,2,0.36198
3,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,3,serious,1,serious,bland,bored,negative,unexcited,serious,NaN,negative,2,the mostly-gray palette makes the infographic ...,y,visual_interest,0,1,4,785.707,burger,negative,3,-0.18446
4,5cc89a457ef1c400046e39a7,ryan,stumbaugh,22,computer science,1,1,"Tue, 30 Apr 2019 18:39:38 GMT","Tue, 30 Apr 2019 18:41:40 GMT","Tue, 30 Apr 2019 18:56:12 GMT",0.8,1.0,0.6,1.0,4,serious,3,serious,negative,bland,bored,unexcited,NaN,serious,negative,3,"similar to the last one, this is a much more b...",y,tone,0,1,3,837.279,gaming,negative,4,-0.18446


In [ ]:
.to_csv("")

In [16]:
palette_df.groupby('palette').agg(np.mean) #averages --> could do t-test
palette_df.groupby('palette').agg([np.size, np.mean, np.std])
#print(palette_df.loc[palette_df['palette'] == 'calm'])
strict_acc = []
med_acc = []
best_val = []
med_val = []
paletteNames = ["disturbing", "negative", "serious", "trustworthy", "calm", "playful", "positive", "exciting"];
time = []
emo_valence = []
for i in paletteNames:
    a = palette_df.loc[palette_df['palette'] == i]
    for j in paletteNames:
        b = palette_df.loc[palette_df['palette'] == j]
        if (i != j):
            t2, p2 = stats.ttest_ind(a['strict_accuracy'], b['strict_accuracy'])
            t,p = stats.ttest_ind(a['med_accuracy'], b['med_accuracy'])
            t1,p1 = stats.ttest_ind(a['best_valence_accuracy'], b['best_valence_accuracy'])
            t3,p3= stats.ttest_ind(a['med_valence_accuracy'], b['med_valence_accuracy'])
            t4,p4= stats.ttest_ind(a['time'], b['time'])
            t5,p5= stats.ttest_ind(a['emotion_valence_avg'], b['emotion_valence_avg'])
        
            strict_acc.append([i,j,p2])
            med_acc.append([i,j,p])
            best_val.append([i,j,p1])
            med_val.append([i,j,p3])
            time.append([i,j,p4])
            emo_valence.append([i,j,p5])
            

#nothing is significantly significant .. interesting
med_val
best_val
med_acc
time
palette_df
emo_valence

[['disturbing', 'negative', 0.50207356540436909],
 ['disturbing', 'serious', 0.47907810166892251],
 ['disturbing', 'trustworthy', 0.015995546864561425],
 ['disturbing', 'calm', 0.00021346983609956703],
 ['disturbing', 'playful', 1.7108483721473528e-07],
 ['disturbing', 'positive', 4.5724551239031266e-06],
 ['disturbing', 'exciting', 0.0034349266154732935],
 ['negative', 'disturbing', 0.50207356540436909],
 ['negative', 'serious', 0.85998405799874933],
 ['negative', 'trustworthy', 0.098855032061822792],
 ['negative', 'calm', 0.0028419144837081448],
 ['negative', 'playful', 1.7959746525013486e-06],
 ['negative', 'positive', 3.8657739680084862e-05],
 ['negative', 'exciting', 0.027569768027921937],
 ['serious', 'disturbing', 0.47907810166892251],
 ['serious', 'negative', 0.85998405799874933],
 ['serious', 'trustworthy', 0.21549870636309937],
 ['serious', 'calm', 0.021225206657847925],
 ['serious', 'playful', 0.00021377985173069792],
 ['serious', 'positive', 0.0017782233833922728],
 ['serio

In [17]:
#maybe plot some distributions in seaborn to see if there are any interesting trendlines

In [18]:
#grouping by perception and analyzing

In [19]:
#significant changes in usage for positive / negative light? 

In [20]:
#strengths of perceptions of certain palettes